In [1]:
import pandas as pd
import plotly.express as px
from ortools.sat.python import cp_model

In [2]:
days = range(5)
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
sessions = range(17)
sess_times = ["08:00 - 10:00","08:30 - 10:30","09:00 - 11:00",
             "09:30 - 11:30","10:00 - 12:00","10:30 - 12:30",
             "11:00 - 13:00","11:30 - 13:30","12:00 - 14:00",
             "12:30 - 14:30","13:00 - 15:00","13:30 - 15:30",
             "14:00 - 16:00","14:30 - 16:30","15:00 - 17:00",
             "15:30 - 17:30","16:00 - 18:00"]
tracers = range(4)
tracers_names = ["PIB","AV1451","MK6240","UCB-J"] #PIB, AV1451, MK6240, UCB-J
tracers_colors = ["2","3","4","5"]

In [3]:
# find all the solutions

# Creates the model.
model = cp_model.CpModel()

In [4]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is [0,1] Positive Integers
x = {}
for t in tracers:
    for d in days:
        for s in sessions:
            x[(t,d,s)] = model.NewBoolVar(
                "{}{}{}".format(tracers_names[t],
                                day_names[d],
                                sess_times[s]))
            
            #x[(t,d,s)] = model.NewBoolVar('shift_n%id%is%i' % (t, d, s))
            #x[(t,d,s)] = model.NewBoolVar("x[{},{},{}]".format(d,s,t))
#x = {}
#for s in sessions:
#    for d in days:
#        for t in tracers:
#            x[d,s,t] = model.NewBoolVar("x[{},{},{}]".format(d,s,t))
# old attempt            model.NewIntVar(0,1,"x{}{}{}".format(d,s,t))

In [5]:
# Creates the constraints.
#Each session gets at most 1 tracer
for d in days:
    for s in sessions:
        model.Add(sum(x[(t,d,s)] for t in tracers) <= 1)

In [6]:
#Each tracer gets at least 1 session per week
for t in tracers:
    all_sess = []
    for d in days:
        for s in sessions:
            all_sess.append(x[(t,d,s)])
    model.Add(sum(all_sess) >= 1)

In [7]:
#PIB 3 max per day/batch
t = tracers_names.index("PIB")
for d in days[1:4]:
    p_day = []
    for s in sessions:
        p_day.append(x[(t,d,s)])
        model.Add(sum(p_day) <= 3)

In [8]:
#PIB 1 max per Monday
t = tracers_names.index("PIB")
d = day_names.index("Mon")
p_mon = []
for s in sessions:
    p_day.append(x[(t,d,s)])
model.Add(sum(p_day) <= 1)

In [9]:
#12 per month for AV1451; 3 per week
t = tracers_names.index("AV1451")
a_week = []
for d in days:
    for s in sessions:
        a_week.append(x[(t,d,s)])
model.Add(sum(a_week) <= 3)

In [10]:
#MK6240 max 3 sessions each day
t = tracers_names.index("MK6240")
for d in days:
    m_3 = []
    for s in sessions:
        m_3.append(x[(t,d,s)])
    model.Add(sum(m_3) <= 3)

In [11]:
#UCB-J max 1 session each day
t = tracers_names.index("UCB-J")
for d in days:
    u_1 = []
    for s in sessions:
        u_1.append(x[(t,d,s)])
    model.Add(sum(u_1) <= 1)

In [12]:
#2 hr slots without overlap
for d in days:
    for s in sessions[0:14]:
        overlap = []
        for t in tracers:
            overlap.extend([
                x[(t,d,s)],x[(t,d,s+1)],
                x[(t,d,s+2)],x[(t,d,s+3)]
            ])
        model.Add(sum(overlap) <= 1)

In [13]:
#24 HR gap AV1451
t = tracers_names.index("AV1451")

for d in days[0:4]:
    for s in sessions:
        a_24 = []
        a_24.extend([x[(t,d,sess)] for sess in sessions if sess >= s])#get rest of sessions in sameday
        a_24.extend([x[(t,d+1,sess)] for sess in sessions if sess < s])#get sessions within 24 hours later
        model.Add(sum(a_24) <= 1)

In [14]:
#12:00 earliest sess for AV1451
t = tracers_names.index("AV1451")
a_early = []
for d in days:
    for s in sessions[0:8]:
        a_early.append(x[(t,d,s)])
        #print("x{}{}{}".format(d,s,t)) #sum() = 0
model.Add(sum(a_early) == 0)

In [15]:
#PIB 3 hr gap
t = tracers_names.index("PIB")
for d in days:
    for s in sessions[0:12]:
        p_gap = []
        for num in range(6):
            p_gap.append(x[(t,d,s+num)])
        model.Add(sum(p_gap) <= 1)

[PIBMon08:00 - 10:00(0..1), PIBMon08:30 - 10:30(0..1), PIBMon09:00 - 11:00(0..1), PIBMon09:30 - 11:30(0..1), PIBMon10:00 - 12:00(0..1), PIBMon10:30 - 12:30(0..1)]
[PIBMon08:30 - 10:30(0..1), PIBMon09:00 - 11:00(0..1), PIBMon09:30 - 11:30(0..1), PIBMon10:00 - 12:00(0..1), PIBMon10:30 - 12:30(0..1), PIBMon11:00 - 13:00(0..1)]
[PIBMon09:00 - 11:00(0..1), PIBMon09:30 - 11:30(0..1), PIBMon10:00 - 12:00(0..1), PIBMon10:30 - 12:30(0..1), PIBMon11:00 - 13:00(0..1), PIBMon11:30 - 13:30(0..1)]
[PIBMon09:30 - 11:30(0..1), PIBMon10:00 - 12:00(0..1), PIBMon10:30 - 12:30(0..1), PIBMon11:00 - 13:00(0..1), PIBMon11:30 - 13:30(0..1), PIBMon12:00 - 14:00(0..1)]
[PIBMon10:00 - 12:00(0..1), PIBMon10:30 - 12:30(0..1), PIBMon11:00 - 13:00(0..1), PIBMon11:30 - 13:30(0..1), PIBMon12:00 - 14:00(0..1), PIBMon12:30 - 14:30(0..1)]
[PIBMon10:30 - 12:30(0..1), PIBMon11:00 - 13:00(0..1), PIBMon11:30 - 13:30(0..1), PIBMon12:00 - 14:00(0..1), PIBMon12:30 - 14:30(0..1), PIBMon13:00 - 15:00(0..1)]
[PIBMon11:00 - 13:00(0

In [16]:
#MK6240 only sessions 10 through 16; Tues-Fri
t = tracers_names.index("MK6240")
m_days = []
sess_all = [0,1,2,3,4,5,6,7,8,16]
for d in days[1:5]:
    for s in list(sessions[0:9])+list(sessions[16:18]):
        m_days.append(x[(t,d,s)])
model.Add(sum(m_days) == 0)

In [17]:
#MK6240 only sessions 8 through 13; Mon
t = tracers_names.index("MK6240")
d = day_names.index("Mon")
m_mon = []
for s in list(sessions[0:7])+list(sessions[13:18]):
    m_mon.append(x[(t,d,s)])
model.Add(sum(m_mon) == 0)

In [18]:
#UCB-J session 6 or 12 Tues-Fri
t = tracers_names.index("UCB-J")
for d in days[1:5]:
    u_sess = []
    for s in list(sessions[0:5])+list(sessions[6:11])+list(sessions[12:18]):
        u_sess.append(x[(t,d,s)])
    model.Add(sum(u_sess) == 0)

In [19]:
#UCB-J session 14 Mon
t = tracers_names.index("UCB-J")
d = day_names.index("Mon")
u_mon = []
for s in list(sessions[0:13])+list(sessions[14:18]):
    u_mon.append(x[(t,d,s)])
model.Add(sum(u_mon) == 0)

In [20]:
#Require each day to have at least n scans
for d in days:
    daily_sess = []
    for t in tracers:
        for s in sessions:
            daily_sess.append(x[(t,d,s)])
    model.Add(sum(daily_sess) >= 4)

In [21]:
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit
        #self.solutions = {}


    def on_solution_callback(self):      
        self._solution_count += 1
        #Create dataframe to hold results
        results = []
        #results = {}
        #results = [{"Solution": self._solution_count}]
        #print('Solution %i' % self._solution_count)
        for d in self._num_days:
            #Get each day's schedule
            day = ["Day{}".format(d)]
            #day = {}
            #day = [{"Day":d}]
            #for d in range(self._num_days):
            #print('Day %i' % d)
            for n in self._num_nurses:
            #for n in range(self._num_nurses):
                is_working = False
                for s in self._num_shifts:
                #for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        #print('  Nurse %i works shift %i' % (n, s))
                        day.append([d,n,s,1])
                        #day.update({(d,n,s) : 1})
                        #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":1})
                if not is_working:
                    #print('  Nurse {} does not work'.format(n))
                    #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":0})
                    #day.update({(d,n,s) : 0})
                    day.append([d,n,s,0])
            #results.update({d : day})
            results.append(day)
        #self.solutions.update({self._solution_count : results})
        pd.DataFrame(results).to_csv("schedule_solution_{}.csv".format(self._solution_count),index=False)
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count


In [22]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
#Enumerate all solutions
solver.parameters.enumerate_all_solutions = True
#Display the first n solutions
solution_limit = 3
solution_printer = NursesPartialSolutionPrinter(x, tracers,
                                                days, sessions,
                                                solution_limit)
solver.Solve(model, solution_printer)
# Statistics.
print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())


Statistics
  - conflicts      : 215
  - branches       : 1113
  - wall time      : 0.037803 s
  - solutions found: 0


In [23]:
sch_data = pd.DataFrame([
    dict(Session=sess_times[0], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[1], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[2], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[3], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[4], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[5], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[6], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[7], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[8], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[9], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[10], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[11], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[12], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[13], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[14], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[15], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[16], Start=0, Finish=5, Tracer="NA", color="1")
])

#Graph solution
sch = pd.read_csv("schedule_solution_1.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[3]) == 1:
                sch_entry = dict(Session=sess_times[int(ent_l[2])],
                                Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                Tracer=tracers_names[int(ent_l[1])], 
                                 color=tracers_colors[int(ent_l[1])])
                sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()

#fig.show()
file_name = 'scheduling_graph_sol_1'
fig.write_html(f"{file_name}.html")

In [24]:
sch_data = pd.DataFrame([
    dict(Session=sess_times[0], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[1], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[2], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[3], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[4], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[5], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[6], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[7], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[8], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[9], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[10], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[11], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[12], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[13], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[14], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[15], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[16], Start=0, Finish=5, Tracer="NA", color="1")
])

#Graph solution
sch = pd.read_csv("schedule_solution_2.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[3]) == 1:
                sch_entry = dict(Session=sess_times[int(ent_l[2])],
                                Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                Tracer=tracers_names[int(ent_l[1])], 
                                 color=tracers_colors[int(ent_l[1])])
                sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()

#fig.show()
file_name = 'scheduling_graph_sol_2'
fig.write_html(f"{file_name}.html")

In [25]:
sch_data = pd.DataFrame([
    dict(Session=sess_times[0], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[1], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[2], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[3], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[4], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[5], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[6], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[7], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[8], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[9], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[10], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[11], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[12], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[13], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[14], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[15], Start=0, Finish=5, Tracer="NA", color="1"),
    dict(Session=sess_times[16], Start=0, Finish=5, Tracer="NA", color="1")
])

#Graph solution
sch = pd.read_csv("schedule_solution_3.csv",
                  index_col=0).T #transpose to get days as columns
#Add each scheduled scan to calendar
for column in sch:
    day = sch[column]
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[3]) == 1:
                sch_entry = dict(Session=sess_times[int(ent_l[2])],
                                Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                Tracer=tracers_names[int(ent_l[1])], 
                                 color=tracers_colors[int(ent_l[1])])
                sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig.update_yaxes(autorange="reversed")
fig.layout.xaxis.type = 'linear'
for d in fig.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()

#fig.show()
file_name = 'scheduling_graph_sol_3'
fig.write_html(f"{file_name}.html")

In [26]:
#fig["layout"]["xaxis"]["title"] = "Day"

In [27]:
#fig["layout"]["template"]

In [28]:
#fig.to_dict()